# Introduction

This document introduces early users of X4 to pyradarlib4, a python package that gives access to X4 settings and radar data. The user should use the X4Driver class to communicate with the radar as demonstrated below.


> **Warning**

> pyradarlib4 is in beta and is provided to early users of X4. All functionality are subject to change; the X4Driver interface will probably break in the future.

> Feedback is appreciated, send an email to support@xethru.com with subject: X4Driver < brief description>

This Jupyter Notebook is available in the pyradarlib4/doc folder.


# Installation

## Prerequisites

**MCU Drivers**

Install the atmel usb driver package, available here (login required): https://gallery.atmel.com/Products/Details/6272a8fd-68fe-43d8-a990-741878cfe7b6

Alternativelly, the atmel usb driver package is bundled with XeThru Explorer: https://www.xethru.com/community/resources/xethru-explorer-for-windows.5/


**Python**

* Python 2.7
* numpy
* matplotlib
* pyserial

It is recommended to install the python distribution package anaconda from continuum: https://www.continuum.io/downloads which contains most packages needed. pyserial can then be installed from the command line with:

> conda install pyserial

## pyradarlib4 installation

Download the latest version from https://www.xethru.com/community/resources/pyradarlib4.60/. Unzip the folder, open a command line interface, cd to the pyradarlib4 folder and run the installer:

> cd < path-to-folder\>

> python setup.py install

## Test the installation

Start an ipython session with interactive matplotlib and pylab enabled:

> ipython --pylab

The following commands connects to the radar, samples one frame and plots it:

> import pyradarlib4

> r = pyradarlib4.X4Driver()

> plot(r.get_frame_normalized())




#X4Driver Example Usage

This section demonstrates the X4Driver interface. Code blocks starts with "In [n]" and can be run in an ipython session. The radar is connected with an USB cable from the XTMCU01 usb port to the computer.


In [ ]:
import pyradarlib4
from pylab import *
%matplotlib inline

In [ ]:
r = pyradarlib4.X4Driver()

In [ ]:
print "This jupyter notebook was generated with the following X4 sw versions:"
print '#'*30
print ""
print 'pyradarlib4: V',pyradarlib4.__version__
print 'MCU FW:      V', r.radar.get_product()

## Sampling frames

The radar is setup and ready to sample frames. A single normilzed frame is sampled with:

In [ ]:
print r.get_frame_normalized.__doc__

In [ ]:
frame_etsi = r.get_frame_normalized()
# Change transmitter center frequency from the to kcc
r.set_tx_region('kcc')
# Increase PG power for KCC transmitter
r.reg.tx_power = 3
frame_kcc = r.get_frame_normalized()
# we'll use the etsi transmitter for now
r.set_tx_region('eu')
r.reg.tx_power = 2

fs = r.get_fs()
bins = len(frame_etsi)
distx =  np.arange(bins)/fs*3e8/2

In [ ]:
figure(figsize = (20,10))
subplot(211)
plot(frame_etsi, label='etsi')
plot(frame_kcc, label='kcc')
xlabel('Bin #')
ylabel('DACValue (0 - 2047)')
xlim(0,bins)
legend()
grid()
ax = gca().twiny()
ax.set_xlim(distx[0], distx[-1])
title("Sampled frame")
print "One frame consist of %d bins, sampled at %g GHz, corresponding to %g meters"%(bins, fs/1e9, distx[-1])

subplot(212)
psd(frame_etsi, Fs = fs/1e9, label = 'etsi')
psd(frame_kcc, Fs = fs/1e9, label = 'kcc')
ylabel("Power")
_=title("PSD of the sampled frame. Remember that this does not reflect the actual transmitted power.")

## On-chip Downconversion to Baseband and Decimation

X4 can downconvert the data which greatly reduces the amount of data to be processed. The 1536 bins long frame is downconverted and decimated resulting in 188 complex bins.

In [ ]:
r.enable_downconversion()

In [ ]:
frame = r.get_frame_normalized()
fs = r.get_fs()
bins = len(frame)
distx =  np.arange(bins)/fs*3e8/2

In [ ]:
figure(figsize = (20,10))
subplot(211)
plot(real(frame), label='real')
plot(imag(frame), label='imag')
plot(abs(frame), label='abs')

xlabel('Bin #')
ylabel('DACValue (0 - 2047)')
xlim(0,bins)
grid()
legend()
ax = gca().twiny()
ax.set_xlim(distx[0], distx[-1])
title("Sampled frame")
print "One frame consist of %d bins, sampled at %g GHz, corresponding to %g meters"%(bins, fs/1e9, distx[-1])

subplot(212)
psd(frame, Fs = fs/1e9)
ylabel("Power")
_=title("PSD of the sampled frame. Remember that this does not reflect the actual transmitted power.")

## Streaming Frames

A frame streaming mode is available with start_streaming. The following function starts streaming, waits until *n* frames are sampled and return the frames. This is convinient when recording a data set, use the queue object returned by start_streaming to process live data.

In [ ]:
import inspect
print inspect.getsource(r.get_frames_normalized)

## Adjusting Sweep parameters

Some of the default settings are:

In [ ]:
r.close()
r = pyradarlib4.X4Driver()
bins = len(r.get_frame_normalized())
print "\nDefault settings"
print "#"*30
print "Iterations:    %d"%r.get_iterations()
print "PulsesPerStep: %d"%r.get_pps()
print "PRF:           %g MHz"%(r.get_prf()/1e6)
print "PRI:           %g ns"%(1/r.get_prf()*1e9)
print "Frame length:  %g ns"%((bins-1)/r.get_fs()*1e9)

Accepting clipping of the direct coupled TX to RX pulse, the DAC range can be narrowed to some range around the frame mean, for example:

In [ ]:
dacmin = int(frame_etsi.mean() - 100)
dacmax = int(frame_etsi.mean() + 100)
r.set_dacmin(dacmin)
r.set_dacmax(dacmax)

In [ ]:
frame = r.get_frame_normalized()

In [ ]:
figure(figsize=(20,10))
plot(frame_etsi, label = 'Full DAC range')
plot(frame,'r', label = 'Narrow DAC range')
legend()
grid()

Given a desired FPS, PulsesPerSteps can be calculated.

In [ ]:
fps = 50.
pps = 1/((dacmax-dacmin+1)*r.get_iterations()/r.get_prf()*fps)
# Scale by 0.8
pps = int(pps * 0.8)
r.set_pps(pps)
# Test if the desired fps is possible:
frames = r.get_frames_normalized(200, fps)

Let's enable downconvertion and sample a data set. A person walks from behind the sensor, through the office door, to the wall and back:

In [ ]:
r.enable_downconversion()

In [ ]:
from time import sleep
raw_input("Walk towards the radar and back, will start after 5 seconds [enter]")
sleep(5)
frames = r.get_frames_normalized(700, fps)

In [ ]:
figure(figsize=(20,10))
X,Y = np.meshgrid(distx, arange(0,len(frames)/fps,1/fps))
f=np.abs(diff(frames,axis=0))
pcolormesh(X,Y,f, cmap=cm.jet)
xlabel("Distance (m)")
ylabel("Time (s)")
title("")

## Streaming Radar Data to Socket

The module pyradarlib4.apps.socketserver connects to a radar and streams data to a socket. An example of how to read the data in matlab is given. See documentation in the module itself.

Example of how to start the socketserver:

In [ ]:
from pyradarlib4.apps import socketserver
print inspect.getsource(socketserver.main)

A matlab example on how to read data from the socket is given in the module documentation:

In [ ]:
print socketserver.__doc__

In [ ]:
# Close the connection
r.close()